In [1]:
import numpy as np
import astropy as ap
import astropy.coordinates as coord
import astropy.units as un
from astroquery.simbad import Simbad
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Organise the LSP information

Here we read in and reorganise the information about the MeerKAT LSP pointings. We include the RA and DEC of the phase centre of each pointing, the diameter of the primary beam, the source at the phase centre (if there is one), and the name of the LSP.

In [2]:
def get_primary_beam_size(freq, D):
    '''
    Calculate the primary beam size

    Args:
    freq (float): frequency in Hz
    D (float): maximum baseline length
               in m

    Returns:
    A float of the primary beam
    diammeter in degrees
    '''
    # First convert the frequency
    # to wavelength in m
    lamda = 299792458/freq
    # Then calculate the primary
    # beam diammeter
    return np.rad2deg((1.22*lamda)/D)

Read in the information for the LSPs: MALS, ThunderKAT, Fornax, LADUMA, MHONGOOSE and MIGHTEE.

In [ ]:
catalogue_path = '/raid/driessen/Catalogues/'

In [3]:
MALS_raw = np.genfromtxt('{}MALS-Lband-sample.txt'.format(cataloge_path))

# Get the RA and DEC from the array
ras = MALS_raw[:, :3]
decs = MALS_raw[:, 3:6]
# Convert the RA and DEC into the
# required format for astropy SkyCoord
mals_ra = []
mals_dec = []
for r, ra in enumerate(ras):
    mals_ra.append('{0}h{1}m{2}s'.format(int(ra[0]),
                               int(ra[1]),
                               ra[2]))
    mals_dec.append('{0}d{1}m{2}s'.format(int(decs[r][0]),
                                int(decs[r][1]),
                                decs[r][2]))
radec = coord.SkyCoord(mals_ra, mals_dec, frame='icrs')
freq = MALS_raw[:, 7]*1e9
pb = get_primary_beam_size(freq, 13.5)

mals_dict = {'RA(deg)': radec.ra.deg,
             'DEC(deg)': radec.dec.deg,
             'Freq(Hz)':freq,
             'Beam_FWHM_Diameter(deg)': pb}
mals_df = pd.DataFrame(data=mals_dict)
mals_df['LSP'] = 'MALS'

In [4]:
laduma_df = pd.read_csv('{}LADUMA_coordinates.csv'.format(cataloge_path))
laduma_df['Freq(Hz)'] = 1.4e9
laduma_df['LSP'] = 'LADUMA'
laduma_df['Beam_FWHM_Diameter(deg)'] = get_primary_beam_size(1.4e9,
                                                             13.5)

In [5]:
tkt_df = pd.read_csv('{}ThunderKAT_coordinates.csv'.format(cataloge_path))
tkt_df['Freq(Hz)'] = 1.4e9
tkt_df['LSP'] = 'ThunderKAT'
tkt_df['Beam_FWHM_Diameter(deg)'] = get_primary_beam_size(1.4e9,
                                                          13.5)

In [6]:
mhon_raw = np.genfromtxt('{}mhongoose_candidates.txt'.format(cataloge_path),
                         dtype=str)
mhon_coords = coord.SkyCoord(mhon_raw[:, 3], mhon_raw[:, 4], frame='icrs')

mhon_dict = {'RA(deg)': mhon_coords.ra.deg,
             'DEC(deg)': mhon_coords.dec.deg}
mhon_df = pd.DataFrame(data=mhon_dict)
mhon_df['Freq(Hz)'] = 1.4e9
mhon_df['LSP'] = 'MHONGOOSE'
mhon_df['Beam_FWHM_Diameter(deg)'] = get_primary_beam_size(1.4e9,
                                                           13.5)

In [7]:
fornax_raw = np.genfromtxt(('{}Fornax_mfs_pointingcentres_'
                            '2018_radec.txt').format(cataloge_path))
fornax_coords = coord.SkyCoord(fornax_raw,
                               unit=(un.deg, un.deg))
fornax_dict = {'RA(deg)': fornax_coords.ra.deg,
               'DEC(deg)': fornax_coords.dec.deg}
fornax_df = pd.DataFrame(data=fornax_dict)
fornax_df['Freq(Hz)'] = 1.4e9
fornax_df['LSP'] = 'Fornax'
fornax_df['Beam_FWHM_Diameter(deg)'] = get_primary_beam_size(1.4e9,
                                                             13.5)

In [8]:
mightee_raw = np.genfromtxt('{}MIGHTEE_pointings.tsv'.format(cataloge_path),
                            dtype=str)
mightee_info = []
for row in mightee_raw:
    if row[0] == 'COSMOS':
        mightee_info.append([float(row[1]), float(row[2]), 1.4e9])
    elif 'CDFS' in row[0]:
        mightee_info.append([float(row[1]), float(row[2]), 1.4e9])
        mightee_info.append([float(row[1]), float(row[2]), 0.85e9])
        mightee_info.append([float(row[1]), float(row[2]), 2.125e9])
    else:
        mightee_info.append([float(row[1]), float(row[2]), 1.4e9])
mightee_info = np.array(mightee_info)

mightee_coords = coord.SkyCoord(mightee_info[:, 0].astype(float)*un.deg,
                                mightee_info[:, 1].astype(float)*un.deg)

freq = mightee_info[:, 2]
pb = get_primary_beam_size(freq,
                           13.5)

mightee_dict = {'RA(deg)': mightee_coords.ra.deg,
                'DEC(deg)': mightee_coords.dec.deg,
                'Freq(Hz)':freq,
                'Beam_FWHM_Diameter(deg)': pb}
mightee_df = pd.DataFrame(data=mightee_dict)
mightee_df['LSP'] = 'MIGHTEE'

In [9]:
# Read in all of the known pulsars as
# a proxy for MeerTIME pointing positions
meerTime_raw = np.genfromtxt('{}formatted_ATNF_psrs.csv'.format(cataloge_path),
                             delimiter=',', dtype=str)

meertime_coords = coord.SkyCoord(meerTime_raw[:, 0].astype(float)*un.deg,
                                 meerTime_raw[:, 1].astype(float)*un.deg)

meertime_dict = {'RA(deg)': meertime_coords.ra.deg,
                 'DEC(deg)': meertime_coords.dec.deg}
meertime_df = pd.DataFrame(data=meertime_dict)
meertime_df['Freq(Hz)'] = 1.4e9
meertime_df['LSP'] = 'pulsars'
meertime_df['Beam_FWHM_Diameter(deg)'] = get_primary_beam_size(1.4e9,
                                                               13.5)

Combine the data frames for all of the LSPs

In [10]:
dataframes = [mals_df, laduma_df, tkt_df, mhon_df, fornax_df, meertime_df, mightee_df]

combined_df = pd.concat(dataframes, join='inner', ignore_index=True)

Get the name of the source at the phase centre of each pointing (if there is one). This is just some nice extra information to have.

In [11]:
all_coords = coord.SkyCoord(combined_df['RA(deg)']*un.deg,
                            combined_df['DEC(deg)']*un.deg)
centre_source = []

# Divide the sources up into chunks,
# otherwise astroquery will chuck
# a hissy fit
for s, starts in enumerate(np.arange(0, 4)):
    start = starts * 1000
    end = start + 1000
    coords = all_coords[start:end]

    # Use the coordinates to search for nearby
    # sources for each source
    result_table = Simbad.query_region(coords, radius=2.*un.arcsec)
    # Match the results to the sources
    try:
        result_coords = coord.SkyCoord(list(result_table['RA']),
                                       list(result_table['DEC']),
                                       unit=(un.hourangle, un.deg))

        for c, coo in enumerate(coords):
            seps = coo.separation(result_coords)

            if np.nanmin(seps.deg) < 2./60./60.:
                centre_source.append(result_table[np.nanargmin(seps.deg)]['MAIN_ID'].decode('UTF-8'))
            else:
                centre_source.append('')
    except (TypeError, KeyError) as e:
        # Add a space for any source that doesn't
        # have any SIMBAD matches
        print('No SIMBAD matches within 2 asec: ', start, end)
        for c, coo in enumerate(coords):
            centre_source.append('')
combined_df['PhaseCentreSource'] = centre_source

Save the data frame.

In [12]:
combined_df.to_csv('{}MeerKAT_LSP_AllPointings.csv'.format(cataloge_path),
                   index=False)